In [ ]:
%matplotlib inline
import pandas as pd
import operator
import psycopg2
import pylab
import numpy as np
import datetime
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
from scipy.stats.stats import pearsonr
import matplotlib.lines as mlines
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from statsmodels.tsa import stattools
import statsmodels.api as sm
import scipy
import random
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
import re
from collections import OrderedDict
import statsmodels.api as sm
from scipy import stats
import statsmodels
from statsmodels.graphics.api import qqplot

mpl.rcdefaults()
pd.options.display.mpl_style = 'default'

In [ ]:
# Read database parameters from default_profile
dbitems = {}
with open('default_profile') as f:
    for line in f.readlines():
        item = line.split(" ")[1].split("=")
        dbitems[item[0]] = item[1].strip()
        
# Connect to database with psycopg2
try:
    conn = psycopg2.connect("dbname='%s' user='%s' host='%s' password='%s'"%(dbitems['PGDATABASE'],dbitems['PGUSER'],dbitems['PGHOST'],dbitems['PGPASSWORD']))
except:
    print "Unable to connect to the database"
    
# Connect to database with sqalchemy
conn_sqlalch = create_engine('postgresql+psycopg2://%s:%s@%s/%s'%(dbitems['PGUSER'],dbitems['PGPASSWORD'],dbitems['PGHOST'],dbitems['PGDATABASE']))

# Code

In [ ]:
#input dataframe
cad_df = pd.read_sql_query("SELECT i_eventnumber, iiu_tdispatch, iiu_tclear, pun_unitid FROM clean_csvs.dbo_rfirehouseapparatus", conn)

In [ ]:
#get year
cad_df['year'] = cad_df['iiu_tdispatch'].apply(lambda x: x.year)

In [ ]:
#subset to only 2014 - 2016
cad_df = cad_df[cad_df['year'] >= 2014]

In [ ]:
#get only medic with two numbers
medic_df = cad_df[cad_df['pun_unitid'].apply(lambda x: x[0] == 'M')]
medic_df = medic_df[medic_df['pun_unitid'].apply(lambda x: len(x) == 3)]

In [ ]:
medic_df.index = range(len(medic_df))
medic_df = medic_df.sort('iiu_tdispatch')

In [ ]:
i = 0
for item in medic_df['iiu_tdispatch']:
    if item.hour < 7:
        i += 1
    else:
        break
medic_df = medic_df[i:]

In [ ]:
init = datetime.datetime(2013,1,1,7,0,0)
medic_df['shift'] = medic_df['iiu_tdispatch'].apply(lambda x: int((x-init).total_seconds()/3600/24 + 1))

In [ ]:
#get dictionary of mt units
dict_units = {}
for item in set(medic_df['pun_unitid']):
    dict_units[item] = []

gb_shift = medic_df.groupby('shift')
for name,group in gb_shift:
    gb_unit = group.groupby('pun_unitid')
    for name2, group2 in gb_unit:
        dict_units[name2].append(len(group2))

In [ ]:
#get median runs per medic
median_tups = []
for item in dict_units.keys():
    median_tups.append((item, np.median(dict_units[item])))
    

 

In [ ]:
median_tups = sorted(median_tups, key = lambda x: x[1])

In [ ]:
#plot median runs per medic units
mpl.rcdefaults()
y_pos = np.arange(len(dict_units))
 
plt.bar(y_pos, [i[1] for i in median_tups], align='center')
plt.axhline(y = 8, color = 'r' ,linewidth = 2)
#plt.axhline(y = 12, color = 'r', linewidth = 2)
plt.xticks(y_pos, [i[0] for i in median_tups], rotation = 90)
plt.ylabel('Number of Runs per Shift')
plt.title('Median Runs Per Medic Unit')

plt.savefig('median.jpg')

In [ ]:
#generate history of dispatch and clear times 
hist_tups = []
for index,item in medic_df.iterrows():
    hist_tups.append((item['pun_unitid'], item['iiu_tdispatch'], 'D'))
    hist_tups.append((item['pun_unitid'], item['iiu_tclear'], 'C'))

In [ ]:
hist_tups = sorted(hist_tups, key = lambda x: x[1])

In [ ]:
units_busy = {}
for item in set(medic_df['pun_unitid']):
    units_busy[item] = 0

In [ ]:
busy_history = []
for item in hist_tups:
    if item[2] == 'C':
        units_busy[item[0]] = 0
    else:
        units_busy[item[0]] = 1
    busy_history.append((item[1], sum(units_busy.values())))

In [ ]:
#Get business for a particular day in each year
year = 2013
month = 4
day = 1

f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.step([i[0] for i in busy_history], [i[1] for i in busy_history])
ax1.set_xlim(datetime.datetime(year,month,day), datetime.datetime(year,month,day+1))
ax1.set_xticklabels([])
ax1.set_title('April 1, 2013')
ax1.set_ylabel("MT U/A")

ax2.step([i[0] for i in busy_history], [i[1] for i in busy_history])
ax2.set_xlim(datetime.datetime(year+1,month,day), datetime.datetime(year+1,month,day+1))
ax2.set_xticklabels([])
ax2.set_title('April 1, 2014')
ax2.set_ylabel("MT U/A")

ax3.step([i[0] for i in busy_history], [i[1] for i in busy_history])
ax3.set_xlim(datetime.datetime(year+2,month,day), datetime.datetime(year+2,month,day+1))
ax3.set_xticklabels([])
ax3.set_title('April 1, 2015')
ax3.set_ylabel("MT U/A")

ax4.step([i[0] for i in busy_history], [i[1] for i in busy_history])
ax4.set_xlim(datetime.datetime(year+3,month,day), datetime.datetime(year+3,month,day+1))
ax4.set_xticklabels([])
ax4.set_title('April 1, 2016')
ax4.set_ylabel("MT U/A")

plt.savefig('BusyMedics.jpg')

In [ ]:
all_busy = [i for i in busy_history if i[1] >= 10]

In [ ]:
plt.plot([i[0] for i in all_busy], [i[1] for i in all_busy])
plt.xticks(rotation = 90)
day = 1
plt.xlim(datetime.datetime(2013,1,1), datetime.datetime(2013,6,2))

In [ ]:
#get business at each time of day
busy_history_morn = [i for i in busy_history if i[0].hour in [7,8,9,10,11,12]]
busy_history_aft = [i for i in busy_history if i[0].hour in [13,14,15,16,17,18]]
busy_history_night = [i for i in busy_history if i[0].hour in [19,20,21,22,23,24]]
busy_history_late = [i for i in busy_history if i[0].hour in [1,2,3,4,5,6]]

In [ ]:
np.median([i[1] for i in busy_history_morn]), np.median([i[1] for i in busy_history_aft]), np.median([i[1] for i in busy_history_night]), np.median([i[1] for i in busy_history_late])